## Exercise #1 - Scraping the pokedex

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.request
from IPython.display import Image
import time

In [2]:
#Step1.1
poke_url= 'https://pokemondb.net/pokedex/all'
poke_page=requests.get(poke_url)
poke_page_soup = BeautifulSoup(poke_page.text, 'html.parser')

poke_cnt= poke_page.content

In [3]:
#for extracting all table rows
poke_table=poke_page_soup.find(name="table", class_="data-table")
poke_table_cmplt=poke_table.find("tbody")
poke_table_rows=poke_table_cmplt.find_all("tr") 

#for extracting number of header rows
poke_table_cmplt=poke_table.find("thead")
poke_table_rowsh=poke_table_cmplt.find_all("tr")

total_rows= len(poke_table_rows)+len(poke_table_rowsh)

print('The total number of rows in the table(including header) are:',str(total_rows))

The total number of rows in the table(including header) are: 927


In [4]:
#step 1.2
poke_table_rows1=poke_table_rows[0]
poke_table_rows1

<tr>
<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><span class="img-fixed icon-pkmn" data-alt="Bulbasaur icon" data-src="https://img.pokemondb.net/sprites/sun-moon/icon/bulbasaur.png"></span></span><span class="infocard-cell-data">001</span></td> <td class="cell-name"><a class="ent-name" href="/pokedex/bulbasaur" title="View pokedex for #001 Bulbasaur">Bulbasaur</a></td><td class="cell-icon"><a class="type-icon type-grass" href="/type/grass">Grass</a><br/> <a class="type-icon type-poison" href="/type/poison">Poison</a></td>
<td class="cell-total">318</td>
<td class="cell-num">45</td>
<td class="cell-num">49</td>
<td class="cell-num">49</td>
<td class="cell-num">65</td>
<td class="cell-num">65</td>
<td class="cell-num">45</td>
</tr>

In [5]:
#Step 1.2.1 The name of the pokemon
poke_table_rows1.find(name="a",class_="ent-name").text


'Bulbasaur'

In [6]:
poke_table_rows1.find(name="a",class_="ent-name")['href']

'/pokedex/bulbasaur'

In [7]:
#Step 1.2.3 The type
poke_table_rows1.find(name="td",class_="cell-icon").text

'Grass Poison'

In [8]:
#Step 1.2.4 The total points
poke_table_rows1.find(name="td",class_="cell-total").text


'318'

In [9]:
# Step 1.2.5 In a single list:  ID Number, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed 
poke_list=[]
for i in range(7):
    poke_list.append(poke_table_rows1.find_all(name="td",class_="cell-num")[i].text)
print(poke_list)   

['001', '45', '49', '49', '65', '65', '45']


In [10]:
#Step 1.3 Define a function that takes in a row of the pokedex table and returns it as a DataFrame with a single row.

def pokedataframe(n):
    poke_rows_list=[]
    poke_rows_list.append(poke_table_rows[n].find_all(name="td",class_="cell-num")[0].text)
    poke_rows_list.append(poke_table_rows[n].find(name="a",class_="ent-name").text)
    poke_rows_list.append(poke_table_rows[n].find(name="td",class_="cell-icon").text)
    poke_rows_list.append(poke_table_rows[n].find(name="td",class_="cell-total").text)
    r=poke_table_rows[n].find_all(name="td",class_="cell-num")
    for j in range(1,7):
        poke_rows_list.append(r[j].text)               
    poke_rows_list.append(poke_table_rows[n].find(name="a",class_="ent-name")['href'])
    pokebox=pd.DataFrame(poke_rows_list,index=['ID Number','Name','Type','Total points','HP','Attack','Defense','Sp. Atk','Sp. Def' ,'Speed','url'])
    pokebox=pokebox.T
    return pokebox              

In [11]:
pokedataframe(8)

,ID Number,Name,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,url
0,006,Charizard,Fire Flying,634,78,104,78,159,115,100,/pokedex/charizard


In [12]:
pokedfs=[]
for i in range(926):
    a=pokedataframe(i)
    pokedfs.append(a)
poke_df = pd.concat(pokedfs)
#poke_df  

In [13]:
print(type(poke_df))

<class 'pandas.core.frame.DataFrame'>


## Exercise 2   #Cleaning the Pokedex

In [14]:
# Step 2.1 Make the ID number the first column in the DataFrame 
cols = ['ID Number','Total points','Attack','Defense','Speed']
poke_df[cols] = poke_df[cols].astype(int)
poke_df=poke_df.set_index('ID Number') 


In [15]:
#poke_df

In [16]:
#Step 2.2 Create 18 dummy variables for each type of pokemon

new_pokedf=poke_df.Type.str.split(' ', expand=True).stack()
new_pokedf1=pd.get_dummies(new_pokedf, prefix='Type').groupby(level=0).sum()
new_pokedf1=new_pokedf1.drop(columns=['Type_'])
pokemon_df=pd.concat([poke_df,new_pokedf1], axis=1)
pokemon_df

,Name,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,url,...,Type_Ghost,Type_Grass,Type_Ground,Type_Ice,Type_Normal,Type_Poison,Type_Psychic,Type_Rock,Type_Steel,Type_Water
ID Number,,,,,,,,,,,,,,,,,,,,,
1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45,/pokedex/bulbasaur,...,0,1,0,0,0,1,0,0,0,0
2,Ivysaur,Grass Poison,405,60,62,63,80,80,60,/pokedex/ivysaur,...,0,1,0,0,0,1,0,0,0,0
3,Venusaur,Grass Poison,525,80,82,83,100,100,80,/pokedex/venusaur,...,0,2,0,0,0,2,0,0,0,0
3,Venusaur,Grass Poison,625,80,100,123,122,120,80,/pokedex/venusaur,...,0,2,0,0,0,2,0,0,0,0
4,Charmander,Fire,309,39,52,43,60,50,65,/pokedex/charmander,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,Stakataka,Rock Steel,570,61,131,211,53,101,13,/pokedex/stakataka,...,0,0,0,0,0,0,0,1,1,0
806,Blacephalon,Fire Ghost,570,53,127,53,151,79,107,/pokedex/blacephalon,...,1,0,0,0,0,0,0,0,0,0
807,Zeraora,Electric,600,88,112,75,102,80,143,/pokedex/zeraora,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#Step 2.3 Remove duplicate values of pokemon based on the URL.Keep the first observation in the case of a duplicate.
deduplipoke_df=poke_df.drop_duplicates(subset='url', keep='first')
deduplipoke_df
#Step 2.3 The number of rows in the deduplicated dataset.
print("The number of rows in the deduplicated dataset: ", str(len(deduplipoke_df)))

The number of rows in the deduplicated dataset:  809


In [19]:
# Step 2.4 Add a dummy variable to the DataFrame called "sample" that tags every 4th pokemon to be included in the sample.
#for col in pokemon:
    #print(col)    
#df = pokemon[pokemon.index % 4 == 0]
sample=[]
for i in deduplipoke_df.index:
    if i%4==0:
        sample.append(1)
    else:
        sample.append(0)       
deduplipoke_df['Sample']=sample
deduplipoke_df.head(10)    

C:\Users\Umang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Name,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,url,Sample
ID Number,,,,,,,,,,,
1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45,/pokedex/bulbasaur,0
2,Ivysaur,Grass Poison,405,60,62,63,80,80,60,/pokedex/ivysaur,0
3,Venusaur,Grass Poison,525,80,82,83,100,100,80,/pokedex/venusaur,0
4,Charmander,Fire,309,39,52,43,60,50,65,/pokedex/charmander,1
5,Charmeleon,Fire,405,58,64,58,80,65,80,/pokedex/charmeleon,0
6,Charizard,Fire Flying,534,78,84,78,109,85,100,/pokedex/charizard,0
7,Squirtle,Water,314,44,48,65,50,64,43,/pokedex/squirtle,0
8,Wartortle,Water,405,59,63,80,65,80,58,/pokedex/wartortle,1
9,Blastoise,Water,530,79,83,100,85,105,78,/pokedex/blastoise,0


## Exercise 3

In [20]:
#poke_img_url="https://img.pokemondb.net/artwork/bulbasaur.jpg"
#poke_img_name=poke_img_url.split("/")[-1]
#carousel_div = poke_page_soup.find(name="div", class_="grid-col")
#poke_img_url = carousel_div.find('img')['src']


In [21]:
#Step 3.1
def poke_img_display(poke_id):
    pk=poke_id
    url='https://pokemondb.net'+str(deduplipoke_df['url'][pk])
    link=requests.get(url)
    text=BeautifulSoup(link.text,'html.parser')
    images = text.find('img', {'src':re.compile('.jpg')})
    print(url)
    img_url=images.get('src')
    return Image(url=img_url)



In [22]:
poke_img_display(4)
poke_img_display(8)

https://pokemondb.net/pokedex/charmander
https://pokemondb.net/pokedex/wartortle


In [23]:
# Step 3.2
#Extract the location table to extract all of the tables from the Bulbasaur page
img_url='https://pokemondb.net/pokedex/bulbasaur'
#Using the given code
tables = pd.read_html(requests.get(img_url, headers={'User-agent': 'Mozilla/5.0'}).text)
tables[-2]

,0,1
0,RedBlue,Pallet Town
1,Yellow,Cerulean City
2,GoldSilverCrystal,Trade/migrate from another game
3,RubySapphire,Trade/migrate from another game
4,FireRedLeafGreen,Pallet Town
5,Emerald,Trade/migrate from another game
6,DiamondPearlPlatinum,Trade/migrate from another game
7,HeartGoldSoulSilver,Pallet Town
8,BlackWhiteBlack 2White 2,Trade/migrate from another game
9,XY,Lumiose City


In [24]:
# Step 3.3
#Transpose the DataFrame 
table=tables[-2].T
#Each column is a video game and each row/cell is the
table.columns = table.iloc[0]
table=table.drop(table.index[0])
#Each column is the location where you find Bulbasaur in that game
table_new = table.rename(index={1: 'Location'})
table_new


,RedBlue,Yellow,GoldSilverCrystal,RubySapphire,FireRedLeafGreen,Emerald,DiamondPearlPlatinum,HeartGoldSoulSilver,BlackWhiteBlack 2White 2,XY,Omega RubyAlpha Sapphire,SunMoon,Ultra SunUltra Moon,Let's Go PikachuLet's Go Eevee,SwordShield
Location,Pallet Town,Cerulean City,Trade/migrate from another game,Trade/migrate from another game,Pallet Town,Trade/migrate from another game,Trade/migrate from another game,Pallet Town,Trade/migrate from another game,Lumiose City,Trade/migrate from another game,Trade/migrate from another game,Route 2,"Cerulean City, Viridian Forest",Location data not yet available


In [25]:
#Step 3.4
#Extract the location table and transpose it for all the pokemon in the sample
new_deduplipoke_df1 = pd.DataFrame(deduplipoke_df)
new_deduplipoke_df2 = deduplipoke_df.loc[deduplipoke_df['Sample'] == 1]
new_pokemon_list = []
new_poke_list = []
for name in new_deduplipoke_df2['url']:
    time.sleep(0.1)
    #remove Meltan from sample
    if name == '/pokedex/Meltan':
        break
    url = 'https://pokemondb.net/' + str(name)
    tables = pd.read_html(requests.get(url, headers = {'User-agent': 'Mozilla/5.0'}).text)[-2]
    tables = tables.T
    tables.columns = tables.iloc[0]
    tables = tables[1:] 
    print(table)
    if 'XY' in tables.columns:
        new_deduplipoke_df3 = pd.DataFrame({'url': name, 'XY' : tables['XY']})
        new_deduplipoke_df4 = pd.DataFrame({'url': url, 'XY' : tables['XY']})
        new_pokemon_list.append(new_deduplipoke_df3)
        new_poke_list.append(new_deduplipoke_df4)
new_deduplipoke_df_final1 = pd.concat(new_pokemon_list)
new_deduplipoke_df_final2 = pd.concat(new_poke_list)
new_deduplipoke_df_final1

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

...

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0                     RubySapphire FireRedLeafGreen  \
1  Trade/migrate from another game      Pallet Town   

0                          Emerald             DiamondPearlPlatinum  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 HeartGoldSoulSilver         BlackWhiteBlack 2White 2            XY  \
1         Pallet Town  Trade/migrate from another game  Lumiose City   

0         Omega RubyAlpha Sapphire                          SunMoon  \
1  Trade/migrate from another game  Trade/migrate from another game   

0 Ultra SunUltra Moon  Let's Go PikachuLet's Go Eevee  \
1             Route 2  Cerulean City, Viridian Forest   

0                      SwordShield  
1  Location data not yet available  
0      RedBlue         Yellow                GoldSilverCrystal  \
1  Pallet Town  Cerulean City  Trade/migrate from another game   

0  

KeyboardInterrupt: 

## Exercise 4.1

In [40]:
#Step 4.1 Create a table that shows the average attack and defense for each type
new_pokedf1.columns
pokedex_strength = []
for Type_of_pokemon in new_pokedf1:
    attack = pokemon_df[pokemon_df[Type_of_pokemon]==1]['Attack'].mean()
    defense = pokemon_df[pokemon_df[Type_of_pokemon]==1]['Defense'].mean()
    pokedex_strength.append([Type_of_pokemon, attack , defense])
Average_df = pd.DataFrame(pokedex_strength, columns = ['Type_of_pokemon', 'Average_Attack', 'Average_Defense'])
Average_df 

,Type_of_pokemon,Average_Attack,Average_Defense
0,Type_Bug,65.555556,69.097222
1,Type_Dark,88.535714,65.053571
2,Type_Dragon,95.959184,82.571429
3,Type_Electric,74.076923,69.211538
4,Type_Fairy,60.640000,70.520000
5,Type_Fighting,102.563636,74.054545
6,Type_Fire,77.961538,71.884615
7,Type_Flying,75.360000,68.720000
8,Type_Ghost,80.903846,82.269231
9,Type_Grass,71.561224,72.500000


In [48]:
Average_df['Average_Attack'].max()
Average_df['Average_Attack'].min()
Average_df['Average_Defense'].max()
Average_df['Average_Defense'].min() 
print(Average_df[['Type_of_pokemon']][Average_df.Average_Attack ==Average_df.Average_Attack.max()],Average_df['Average_Attack'].max())
print(Average_df[['Type_of_pokemon']][Average_df.Average_Attack ==Average_df.Average_Attack.min()],Average_df['Average_Attack'].min())
print(Average_df[['Type_of_pokemon']][Average_df.Average_Defense ==Average_df.Average_Defense.max()],Average_df['Average_Defense'].max())
print(Average_df[['Type_of_pokemon']][Average_df.Average_Defense ==Average_df.Average_Defense.min()],Average_df['Average_Defense'].min())

  Type_of_pokemon
5   Type_Fighting 102.56363636363636
  Type_of_pokemon
4      Type_Fairy 60.64
   Type_of_pokemon
15       Type_Rock 109.5
   Type_of_pokemon
12     Type_Normal 57.95283018867924


##### The type of pokemon that has the highest Average attack strength is Fighting
##### The type of pokemon that has the lowest  Average attack strength is Fairy
##### The type of pokemon that has the highest Defence attack strength is Rock
##### The type of pokemon that has the lowest Defence attack strength is Normal

In [29]:
# Step 4.2 Join the pokedex data to the location DataFrame
poke_df_final=new_deduplipoke_df2.merge(new_deduplipoke_df_final1)
poke_df_final
xy=poke_df_final.groupby(by='XY').mean()
xy

,Total points,Attack,Defense,Speed,Sample
XY,,,,,
Ambrette Town,355.500000,93.000000,77.500000,45.750000,1
Breed Electrode,330.000000,30.000000,50.000000,100.000000,1
Breed Gurdurr/Conkeldurr,305.000000,80.000000,55.000000,35.000000,1
Breed Hariyama,237.000000,60.000000,30.000000,25.000000,1
Breed Haunter/Gengar,310.000000,35.000000,30.000000,80.000000,1
Breed Piloswine/Mamoswine,250.000000,50.000000,40.000000,50.000000,1
Breed Ursaring,330.000000,80.000000,50.000000,40.000000,1
Breed Watchog,255.000000,55.000000,39.000000,42.000000,1
Cyllage City,510.000000,85.000000,200.000000,30.000000,1


In [57]:
#xy.index
#xy['Total points'].max()
xy[['Total points']].idxmax()
print("The location with highest average total point score is: ", xy[['Total points']].idxmax() )

The location with highest average total point score is:  Total points    Sea Spirit's Den, Roaming Kalos
dtype: object


####  The location with highest average total point score is: Sea Spirit's Den, Roaming Kalos